### Topic Modeling - Latent Dirichlet Allocation
토픽 모델링은 Document의 모임에서 발생하는 추상적인 주제(토픽)를 발견하기 위한 통계 모델입니다.

텍스트 문서에서 일반적으로 사용되기도 하지만, 요즘은 소셜 미디어 분석에서의 토픽 모델링도 연구되는 추세입니다.

가장 많이 사용되는 알고리즘 중 하나는 Latent Dirichlet Allocation(잠재 디리클레 할당, LDA)이며, 아래 참고사항을 확인해보세요.

1. 토픽의 개수를 의미하는 k를 결정하려면 perplexity와 elbow method를 사용합니다.
2. 모델을 평가하기 위해서 Topic Coherence(주제 일관성)을 사용합니다.

In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim

### Tokenizing 및 Lemmatization 진행하기
- Tokenizing은 문장을 단어 단위로 분할하는 행위를 일컫습니다.
- Lemmatization은 표제어 추출로, 분할된 단어가 독립적인 형태소(기초 단위 단어)가 아니라면, 어간+접사 형태로 분할하여 독립된 형태소로 만들어줍니다.

In [2]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
eng_stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [3]:
remove_words = ['data', 'dataset', 'datasets', 'content', 'context', 'acknowledgement', 'inspiration']

In [4]:
df = pd.read_csv("./data/voted-kaggle-dataset.csv")
print(df['Description'].head(3))

0    The datasets contains transactions made by cre...
1    The ultimate Soccer database for data analysis...
2    Background\nWhat can we say about the success ...
Name: Description, dtype: object


In [6]:
texts = []
for i in df['Description'].items():
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    stopped_tokens = [raw for raw in tokens if not raw in eng_stopwords]
    stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]

    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw)==1]
    texts.append(new_lemma_tokens)

print(texts[0])

['contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'cannot', 'provide', 'original', 'feature', 'background', 'information', 'feature', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'feature', 'amount', 'transaction', 'amount', 'feature', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring', 'accuracy', 'using', 'area', 'precision', 'recall', 'curve', 'auprc',

In [7]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### LDA 모델 생성

In [9]:
import pprint

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word=dictionary, passes=20)
pprint.pprint(ldamodel.top_topics(corpus, topn=5))

[([(0.007347865, 'file'),
   (0.006786983, 'information'),
   (0.0061662854, 'number'),
   (0.0058850306, 'time'),
   (0.0053381636, 'contains')],
  -0.8652033121343171),
 ([(0.017021215, 'others'),
   (0.014035083, 'http'),
   (0.013608417, 'world'),
   (0.012983495, 'acknowledgement'),
   (0.012637423, 'research')],
  -0.9952361266519516),
 ([(0.0100625595, 'http'),
   (0.009403293, 'com'),
   (0.008158382, 'image'),
   (0.008010076, 'file'),
   (0.007702989, 'csv')],
  -1.2978171229764295),
 ([(0.013436808, 'state'),
   (0.011708238, 'country'),
   (0.010544258, 'school'),
   (0.010463863, 'student'),
   (0.010194143, 'year')],
  -1.8265382940835686),
 ([(0.025691126, 'model'),
   (0.02236307, 'trained'),
   (0.01841422, 'integer'),
   (0.013678608, 'pre'),
   (0.012668707, 'feature')],
  -1.9744998132390468),
 ([(0.00891476, 'time'),
   (0.008290919, 'weather'),
   (0.008156554, 'station'),
   (0.007930283, 'question'),
   (0.0067745643, 'temperature')],
  -2.1978980748437604),
 ([

In [10]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.220196 -0.006773       1        1  34.109034
11     0.205251 -0.035669       2        1  21.023913
3      0.098802 -0.021368       3        1   6.072968
13     0.138401 -0.026143       4        1   5.945231
12     0.107383 -0.007692       5        1   4.872836
6      0.025404 -0.054570       6        1   4.562668
8      0.060983 -0.027209       7        1   4.163264
5     -0.031618  0.082350       8        1   3.491213
14    -0.194788 -0.337820       9        1   3.267152
2     -0.077804  0.075650      10        1   2.748390
0     -0.055247  0.070844      11        1   2.283180
10    -0.126876  0.058660      12        1   2.226597
9     -0.090791  0.088215      13        1   2.201445
1     -0.157731  0.064430      14        1   2.012068
7     -0.121566  0.077096      15        1   1.020040, topic_info=             Term         Freq        Total Category  logprob  loglift
144           com  1609.000000  1609.000000  Default  30.0000  30.0000
944          name  1755.000000  1755.000000  Default  29.0000  29.0000
286          type  1336.000000  1336.000000  Default  28.0000  28.0000
1039  description  1011.000000  1011.000000  Default  27.0000  27.0000
776    university   866.000000   866.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
1220          age    11.287879   336.388111  Topic15  -5.6793   1.1908
246        people    10.251121   475.890420  Topic15  -5.7756   0.7475
536          year    11.030207  1293.327381  Topic15  -5.7024  -0.1790
92       research     9.503217   611.825170  Topic15  -5.8514   0.4205
205     including     8.762151   319.107023  Topic15  -5.9326   0.9902

[1017 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
20548     14  0.789062        _f
20560     14  0.945382        _m
6496       1  0.261620        ab
6496       8  0.713508        ab
512        3  0.670669  accident
...      ...       ...       ...
3195       1  0.392997      zone
3195       4  0.068347      zone
3195       5  0.034174      zone
3195       7  0.034174      zone
3195      12  0.461344      zone

[2779 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 12, 4, 14, 13, 7, 9, 6, 15, 3, 1, 11, 10, 2, 8])